# Тюториал 1.  Классификация текстов с использование векторных представлений слов

Цель данного тюториала -- познакомить участников с применением **DeepPavlov** для классификации текстов.

Тюториал имеет следующую структуру:

1. [Установка зависимостей и библиотеки](#Установка-зависимостей-и-библиотеки)

2. [Скачивание датасета](#Скачивание-датасета)

3. [Dataset Reader](#Dataset-Reader): [docs link](https://deeppavlov.readthedocs.io/en/latest/apiref/dataset_readers.html)

4. [Dataset Iterator](#Dataset-Iterator): [docs link](https://deeppavlov.readthedocs.io/en/latest/apiref/dataset_iterators.html)

5. [Preprocessor](#Preprocessor): [docs link](https://deeppavlov.readthedocs.io/en/latest/components/data_processors.html)

6. [Tokenizer](#Tokenizer): [docs link](https://deeppavlov.readthedocs.io/en/latest/components/data_processors.html)

7. [GloVe Embedder](#Embedder): [docs link](https://deeppavlov.readthedocs.io/en/latest/components/data_processors.html)
[pre-trained embeddings link](https://deeppavlov.readthedocs.io/en/latest/intro/pretrained_vectors.html)

8. [Vocabulary of classes](#Vocabulary-of-classes)

9. [Keras Classifier](#Classifier): [docs link](https://deeppavlov.readthedocs.io/en/latest/components/classifiers.html)

## Установка зависимостей и библиотеки

Начнем с установки библиотеки `DeepPavlov` и дополнительных зависимостей для использвоания нейросетевой модели классификации на `Keras` с `TensorFlow` бэкендом.

In [ ]:
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install intents_snips

## Скачивание датасета

Данный тюториал использует датасет Stanford Sentiment Treebank (SST), описанный в [статье](https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf).

Датасет содержит набор неразмеченных предложений, разделенных на тренировочную, валидационную и тестовую выборку, а также набор фраз, каждой из которых сопоставлено значение тональности от 0 до 1.
Большинство предложений содержатся в наборе фраз. 
Соответственно, мы выбрали только те предложения, что размечены по тональности, для каждой выборки. 
Также мы преобразовали значение тональности, разделив интервал от 0 до 1.
Для решения многоклассовой задачи классификации разделим интервал на 5 частей: 0.0-0.2 - очень негативные, 0.2-0.4 - негативные, 0.4-0.6 - нейтральные, 0.6-0.8 - позитивные, 0.8-1.0 - очень позитивные.
Для сведения к двум классам разделим интервал следующим образом: 0.0-0.4 - негативные, 0.6-1.0 - позитивные.

Скачаем архив с датасетом и извлечем его.

In [ ]:
from deeppavlov.core.data.utils import download

download("./stanfordSentimentTreebank.zip", source_url="http://files.deeppavlov.ai/datasets/stanfordSentimentTreebank.zip")

In [ ]:
!unzip stanfordSentimentTreebank.zip

In [ ]:
!ls stanfordSentimentTreebank/

## Dataset Reader

DatasetReader - компонента библиотеки для чтения файлов. `DeepPavlov` содержит несколько различных DatasetReaders, пользователи могут как использовать уже готовый DatasetReader, так и написать свою компоненту для чтения данных, однако стоит учитывать, что данные на выходе должны быть в определенном формате. 

Требования к **DatasetReader**: 
* на выходе метода `read` должен быть словарь с тремя ключами "train", "valid" и "test", 
* каждый элемент словаря - лист соответствующих примеров,
* каждый пример - tuple `(x, y)`, где `x` и `y` также могут быть листами соответствующих входных данных (например, `x` может быть листом из двух входных текстов, а `y` - листом классов, к которому принадлежит пример).

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/dataset_readers.html

In [ ]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader

In [ ]:
reader = BasicClassificationDatasetReader()
data = reader.read(data_path="./stanfordSentimentTreebank", 
                   train="train_binary.csv", valid="valid_binary.csv", test="test_binary.csv",
                   x="text", y="binary_label")

In [ ]:
data.keys()

Для задачи классификации по умолчанию каждому примеру ставится в соответствие лист классов. Это сделано для единообразия работы как с многоклассовой классификацией, так и с многолейбловой (когда каждому примеру может соответствовать один или несколько классов).

In [ ]:
data["train"][0]

## Dataset Iterator

DatasetIterator - компонента библиотеки для итерирования по датасету (создания батчей, получения всех примеров). `DeepPavlov` содержит несколько разных DatasetIterators, пользователи могут как использовать уже готовый DatasetIterator, так и написать свою компоненту для итерирования по данным

DatasetIterator должен иметь следующие методы:

* **gen_batches** - метод для генерации батчей входных пар для обучения или инференса нейронной сети. На выходе метода - tuple `(x, y)` из двух листов входных данных `x` и `y`. 
* **get_instances** - метод для получения данных определенного набора ("train", "valid", "test"). На выходе - tuple `(x, y)` всех элементов входных данных этого набора.
* **split** - метод для разделения наборов данных на несколько ("train", "valid", "test").

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/dataset_iterators.html

In [ ]:
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

In [ ]:
iterator = BasicClassificationDatasetIterator(data, seed=42, shuffle=True)

In [ ]:
for batch in iterator.gen_batches(data_type="train", batch_size=13):
    print(batch)
    break

## Preprocessor

Предобработать текст можно, используя различные компоненты либо написав свою.
В данном тюториале воспользуемся самым простым - lower-casing.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/preprocessors.html

In [ ]:
from deeppavlov.models.preprocessors.str_lower import StrLower

In [ ]:
preprocessor = StrLower()

In [ ]:
preprocessor(["The Rock is destined to be the 21st Century 's new `` Conan ''."])

## Tokenizer

Для использования потокенных (пословных) векторных представлений необходимо разделить предобработанный текст на токены (слов и пунктуационные символы).
`DeepPavlov` содержит несколько различных токенизаторов. Пользователи могут выбрать любой.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/tokenizers.html

In [ ]:
from deeppavlov.models.tokenizers.nltk_tokenizer import NLTKTokenizer

In [ ]:
tokenizer = NLTKTokenizer()

In [ ]:
tokenizer(["The Rock is destined to be the 21st Century 's new `` Conan ''."])

## Embedder

В данном тюториале используем недообучаемые векторные представления GloVe. 
Все компоненты для получения векторных представлений текстов унифицированы, поэтому пользователи могут использовать любые векторные представления из предложенных в документации: GloVe, fastText, ELMo.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/embedders.html

Скачаем файл с векторными представлениями GloVe. Данный файл был скачан [отсюда](https://nlp.stanford.edu/projects/glove/), но по ссылке скачивается еще несколько других файлов (в итоге более 800 Mb). Для того, чтобы сберечь наше время, скачаем файл с векторными представлениями с `DeepPavlov` (скачивает 350 Mb).

In [ ]:
from deeppavlov.core.data.utils import download

download("./glove.6B.100d.txt", source_url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt")

Now we can define GloVeEmbedder. Parameter `pad_zero` which is set to `True` determines whether to pad embedded batch of tokens to the longest sample length.

In [ ]:
from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

embedder = GloVeEmbedder(load_path="./glove.6B.100d.txt", 
                         pad_zero=True  # means whether to pad up to the longest sample in a batch
                        )

In [ ]:
embedder.dim

In [ ]:
embedder(tokenizer(preprocessor(["The Rock is destined to be the 21st Century 's new 'Conan'.",
                                 "The Rock is a new 'Conan'."])))

In [ ]:
embedder(tokenizer(preprocessor(["The Rock is destined to be the 21st Century 's new 'Conan'.",
                                 "The Rock is a new 'Conan'."]))).shape

## Vocabulary of classes

По умолчанию каждый класс считывается и хранится в виде строки.
Поэтому мы должны просмотреть все встречающиеся в датасете классы, собрать по ним словарь, а затем преобразовать классы в индексы.

Нейросетевые классификаторы принимают one-hot представление распределения по классам, а возвращают распределение вероятностей принадлежности классам.
Чтобы получить one-hot представление классов, необходимо также преобразовать полученные из словаря индексы классов.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/core/data.html

In [ ]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

In [ ]:
vocab = SimpleVocabulary(save_path="./binary_classes.dict")

In [ ]:
iterator.get_instances(data_type="train")

In [ ]:
vocab.fit(iterator.get_instances(data_type="train")[1])

In [ ]:
list(vocab.items())

In [ ]:
vocab(["positive", "positive", "negative"])

In [ ]:
vocab([0, 0, 1])

**One-hotter**

Компонента для преобразования индексов в one-hot представление.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/preprocessors.html

In [ ]:
from deeppavlov.models.preprocessors.one_hotter import OneHotter

In [ ]:
one_hotter = OneHotter(depth=vocab.len, 
                       single_vector=True  # means we want to have one vector per sample
                      )

In [ ]:
one_hotter(vocab(["positive", "positive", "negative"]))

**Converting from probability to labels**

Так как нейросетевые модели возвращают вероятности распределения по классам, поэтому нам необходимо также преобразовать вектора вероятностей в классы.
Для этого используем `Proba2Labels` компоненту для преобразования вероятнсотей в индексы, а далее словарь по классам для преобразования индексов в текстовые назвнаия классов.

`Proba2Labels` компонента имеет несколько режимов:
* если `max_proba`=true, возвращает индексы с наибольшей вероятностью,
* если задано значение `confident_threshold` (от 0 до 1), возвращает индексы, вероятность принадлежности к классам которых выше заданного порога,
* если задано целочисленное значение `top_n`, возвращает `top_n` индексов с наибольшими вероятностями.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/preprocessors.html

In [ ]:
from deeppavlov.models.classifiers.proba2labels import Proba2Labels

prob2labels = Proba2Labels(max_proba=True)

In [ ]:
vocab.len

In [ ]:
prob2labels([[0.6, 0.4], 
             [0.2, 0.8],
             [0.1, 0.9]])

In [ ]:
vocab(prob2labels([[0.6, 0.4], 
                   [0.2, 0.8],
                   [0.1, 0.9]]))

## Classifier

`DeepPavlov` содержит несколько различных моделей для классификации текстов: классификаторы `sklearn`, нейросетевые модели на `Keras` с `TensorFlow` бэкендом, классификатор на основе BERT  архитектуры на `TensorFlow`.
Данный тюториал демонстрирует, как построить нейросетевой классификатор неглубокой широкой свёрточной сети (shallow and wide Convolutional Neural Network) на `Keras`. 

`KerasClassificationModel` - класс-компонента, строящий `Keras` классификатор, а архитектура сети задается в отдельном методе класса.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/classifiers.html

In [ ]:
from keras.layers import Input, Dense, Activation, Dropout, Flatten, GlobalMaxPooling1D
from keras import Model

from deeppavlov.models.classifiers.keras_classification_model import KerasClassificationModel
from deeppavlov.metrics.accuracy import sets_accuracy

In [ ]:
model = KerasClassificationModel(
    filters_cnn=256,
    kernel_sizes_cnn=[3,5,7],
    dropout_rate=0.2,
    dense_size=100,
    save_path="./cnn_model_v0", 
    load_path="./cnn_model_v0", 
    embedding_size=embedder.dim,
    n_classes=vocab.len,
    model_name="cnn_model",  # HERE we put our new network-method name
    optimizer="Adam",
    learning_rate=0.001,
    learning_rate_decay=0.001,
    loss="categorical_crossentropy")

In [ ]:
# Method `get_instances` returns all the samples of particular data field
x_valid, y_valid = iterator.get_instances(data_type="valid")
# You need to save model only when validation score is higher than previous one.
# This variable will contain the highest accuracy score
best_score = 0.
patience = 2
impatience = 0

# let's train for 10 epochs
for ep in range(10):
    
    for x, y in iterator.gen_batches(batch_size=64, 
                                     data_type="train", shuffle=True):
        x_embed = embedder(tokenizer(preprocessor(x)))
        y_onehot = one_hotter(vocab(y))
        model.train_on_batch(x_embed, y_onehot)
        
    y_valid_pred = model(embedder(tokenizer(preprocessor(x_valid))))
    score = sets_accuracy(y_valid, vocab(prob2labels(y_valid_pred)))
    print("Epochs done: {}. Valid Accuracy: {}".format(ep + 1, score))
    if score > best_score:
        model.save()
        print("New best score. Saving model.")
        best_score = score    
        impatience = 0
    else:
        impatience += 1
        if impatience == patience:
            print("Out of patience. Stop training.")
            break

In [ ]:
# Let's look into obtained resulting outputs
print("Text sample: {}".format(x_valid[0]))
print("True label: {}".format(y_valid[0]))
print("Predicted probability distribution: {}".format(dict(zip(vocab.keys(), 
                                                               y_valid_pred[0]))))
print("Predicted label: {}".format(vocab(prob2labels(y_valid_pred))[0]))

# Подключение к Telegram

`DeepPavlov` поддерживает подключение моделей и ботов к следующим сервисам:

* [REST API](http://docs.deeppavlov.ai/en/master/intro/features.html#examples-of-some-components)
* [Telegram](http://docs.deeppavlov.ai/en/master/intro/features.html#examples-of-some-components)
* [Amazon Alexa](http://docs.deeppavlov.ai/en/master/devguides/amazon_alexa.html)
* [Microsoft Bot Framework](http://docs.deeppavlov.ai/en/master/devguides/ms_bot_integration.html)
  * Bing, Cortana, Email, Facebook Messenger, Slack, GroupMe, Microsoft Teams, Skype, Telegram, Twilio, Web Chat
* [Yandex Alice](http://docs.deeppavlov.ai/en/master/devguides/yandex_alice.html)

В данном тюториале рассмотрим, как подключить полученную модель к Telegram.

## Подключение из командной строки

`DeepPavlov` использует следующие команды:

Запустить модель в интерфейсе командной строки:
```
python -m deeppavlov interact model_config
```

Поднять REST API:
```
python -m deeppavlov riseapi model_config
```

Подключить к Telegram:
```
python -m deeppavlov interactbot model_config -t <TELEGRAM_TOKEN>
```

Telegram token может быть создан с помощью  @BotFather. Подробнее прочитайте [здесь](https://core.telegram.org/bots#3-how-do-i-create-a-bot).

## Подключение без конфига

В данном тюториале мы не создавали конфиг для модели, поэтому давайте попробуем поднять сервис, также не используя конфиг.

In [2]:
class SentimentPipeline:
    def __init__(self, *args, **kwargs):
        pass

    def __call__(self, input_texts):
        '''
        __call__ method should return responses for each utterance in input_texts
        '''
        
        sentiment_labels = vocab(prob2labels(model(embedder(tokenizer(preprocessor(input_texts))))))
        sentiment_labels = [lab[0] for lab in sentiment_labels]
        
        return sentiment_labels

In [3]:
def generate_config(class_name):
    """generate minimal required DeepPavlov model configuration"""

    config = {
        'chainer': {
            'in': ['x'],
            'out': ['y'],
            'pipe': [
                {
                    'class_name': f'__main__:{class_name}',
                    'in': ['x'],
                    'out': ['y']
                }
            ]
        }
    }
    
    return config

In [4]:
# to interact with CLI
from deeppavlov.core.commands.infer import interact_model
# to interact with Telegram
from deeppavlov.utils.telegram.telegram_ui import interact_model_by_telegram

In [ ]:
interact_model(generate_config('SentimentPipeline'))

In [ ]:
interact_model_by_telegram(generate_config('SentimentPipeline'), 
                           token='YOUR_TOKEN')